In [1]:
from imp import reload
import SubRecommender as sr
import pandas as pd
import json

In [2]:
reload(sr)

<module 'SubRecommender' from 'C:\\Users\\macle\\Desktop\\UPC Masters\\Semester 2\\CI\\SubRecommender\\SubRecommender.py'>

In [2]:
tst = sr.SubRecommender(train_data_file="data/test_reddit_data.json",sequence_chunk_size=51)

In [6]:
with open("data/test_reddit_data.json",'r') as data_file:
    train_data = json.load(data_file)
df = pd.DataFrame(train_data,columns=['sub_seqs','sub_label','seq_length'])
df.head()

,sub_seqs,sub_label,seq_length
0,mrbojenglz,RocketLeague,1.482515e+09
1,mrbojenglz,iamverysmart,1.482513e+09
2,mrbojenglz,evilbuildings,1.482513e+09
3,mrbojenglz,BeAmazed,1.482507e+09
4,mrbojenglz,DadReflexes,1.482441e+09


In [4]:
pd.read_json("data/test_reddit_data.json")

,0,1,2
0,mrbojenglz,RocketLeague,1482515179
1,mrbojenglz,iamverysmart,1482513449
2,mrbojenglz,evilbuildings,1482513372
3,mrbojenglz,BeAmazed,1482506675
4,mrbojenglz,DadReflexes,1482441020
5,mrbojenglz,CompetitiveHS,1482170369
6,mrbojenglz,googlehome,1482165859
7,mrbojenglz,evilbuildings,1481911248
8,mrbojenglz,ArianaGrande,1481906473
9,mrbojenglz,hearthstone,1481901743


In [3]:
tst.load_train_df("data/test_reddit_data.json")

Loading Training Data
Data loaded from disk


,0,1,2
0,mrbojenglz,RocketLeague,1482515179
1,mrbojenglz,iamverysmart,1482513449
2,mrbojenglz,evilbuildings,1482513372
3,mrbojenglz,BeAmazed,1482506675
4,mrbojenglz,DadReflexes,1482441020
5,mrbojenglz,CompetitiveHS,1482170369
6,mrbojenglz,googlehome,1482165859
7,mrbojenglz,evilbuildings,1481911248
8,mrbojenglz,ArianaGrande,1481906473
9,mrbojenglz,hearthstone,1481901743


In [3]:
tst.train("data/test_reddit_data.json")

Building Vocab
Loading Training Data
Data loaded from disk


TypeError: read_json() got an unexpected keyword argument 'columns'

In [4]:
tst.save("models/tst2.json")

TypeError: <tflearn.models.dnn.DNN object at 0x000002DB44C1D128> is not JSON serializable

In [4]:
with open("data/user_comment_sequence_cache.json",'r') as cache_file:
    sequence_cache = json.load(cache_file)

In [5]:
test_list = ['Sinidir', 'Theseuseus', 'Xanthophobia42', 'Rudyard_Hipling', 'kfmush', 'monsterenergyhat']
tst_seqs = [sequence_cache[usr] for usr in test_list]

In [6]:
tst.recommend_subs(tst_seqs)

[['todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'worldnews',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned'],
 ['todayilearned',
  'todayilearned',
  'worldnews',
  'todayilearned',
  'todayilearned'],
 ['todayilearned', 'todayilearned'],
 ['todayilearned'],
 ['worldnews',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'worldnews',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned',
  'todayilearned'],
 ['worldnews']]

In [4]:
"""
Simple example using LSTM recurrent neural network to classify IMDB
sentiment dataset.
References:
    - Long Short Term Memory, Sepp Hochreiter & Jurgen Schmidhuber, Neural
    Computation 9(8): 1735-1780, 1997.
    - Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng,
    and Christopher Potts. (2011). Learning Word Vectors for Sentiment
    Analysis. The 49th Annual Meeting of the Association for Computational
    Linguistics (ACL 2011).
Links:
    - http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf
    - http://ai.stanford.edu/~amaas/data/sentiment/
"""
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.data_utils import to_categorical, pad_sequences

trainX = train['sub_seqs'].values
trainY = train['sub_label'].values
testX = test['sub_seqs'].values
testY = test['sub_label'].values

# Data preprocessing
# Sequence padding
trainX = pad_sequences(trainX, maxlen=max(train['seq_length'].values), value=0.)
testX = pad_sequences(testX, maxlen=max(train['seq_length'].values), value=0.)
# Converting labels to binary vectors
trainY = to_categorical(trainY, nb_classes=tst.vocab_size)
testY = to_categorical(testY, nb_classes=tst.vocab_size)

# Network building
net = tflearn.input_data([None, max(train['seq_length'].values)])
net = tflearn.embedding(net, input_dim=tst.vocab_size, output_dim=128)
net = tflearn.lstm(net, 64, dropout=0.8)
net = tflearn.fully_connected(net, tst.vocab_size, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')

# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True,
          batch_size=32)

Training Step: 150  | total loss: 5.46296
| Adam | epoch: 010 | loss: 5.46296 - acc: 0.0306 | val_loss: 8.65666 - val_acc: 0.0085 -- iter: 470/470
Training Step: 150  | total loss: 5.46296
| Adam | epoch: 010 | loss: 5.46296 - acc: 0.0306 | val_loss: 8.65666 - val_acc: 0.0085 -- iter: 470/470
--


In [15]:
my_tst = test['sub_seqs'].values[0]

In [26]:
prd_check = pad_sequences([my_tst], maxlen=max(train['seq_length'].values), value=0.)
prd_check[0]
ult_prd = model.predict(prd_check[0].reshape((1, -1)))
sum(ult_prd[0])

0.9999999144574758